In [1]:
#preprocessing
import jieba
import pandas as pd

### 1. input train data 

In [2]:
#Input the csv file into the pandas dataframe with specific category'cat,news_content'(10000,2)
#sep = seperator
#pd.dropna drop the data missing some elem
news_data = pd.read_csv("./data/cnews.train.txt",sep = "\t",names=['category','news_content'],encoding='utf-8')
news_data = news_data.dropna()

In [3]:
news_data.shape

(10000, 2)

In [4]:
sport = news_data[news_data.category=='体育']
#split the 'news_content' and get the value part (nparray)and put it into the list
#in this way, we can get the input data and get the category as output target
sport = sport.news_content.values.tolist()

In [5]:
type(sport)

list

In [6]:
home = news_data[news_data.category=='家居']#select category
home = home.news_content.values.tolist()#select value
fashion = news_data[news_data.category=='时尚']
fashion = fashion.news_content.values.tolist()
house = news_data[news_data.category=='房产']
house = house.news_content.values.tolist()
edu = news_data[news_data.category=='教育']
edu = edu.news_content.values.tolist()
politic = news_data[news_data.category=='时政']
politic = politic.news_content.values.tolist()
enter = news_data[news_data.category=='娱乐']
enter = enter.news_content.values.tolist()
game = news_data[news_data.category=='游戏']
game = game.news_content.values.tolist()

In [7]:
home[0]

'仿古地板流行更适合大住宅冬的木地板似乎也受到了“古风”熏染，通体上下透出一股浓浓的“复古味”。据了解，这些复古木地板大多采用手工制作，刻意对木纹进行强化与突出处理，让木地板变得更加生动立体，更富于怀旧气息。也就是目前流行市场的仿古地板。回归自然，个性消费和资源的有效利用是仿古流行的主要因素，让家和自然更接近是现代装修的主题，更具个性的仿古手工制造符合了人无我有的消费个性风格，这是仿古地板风行未来地板市场的理由。有两种客户最有可能成为仿古地板的消费者，一种是欣赏仿古风格，让家更有个性的消费者，这是对地板材种有精确理解，追求厚实、粗犷，原始居家风格的行家，仿古地板一开始好像就是为他们发明的。第二种是时尚消费的盲从者，对仿古地板知其然而不知其所以然，因为时尚，所以购买。选购时要掌握三个要点：1.仿古地板一般不是珍稀名贵地板，材料成本可能比同品种地板略低，因为是手工制造，贵在工艺制造成本；2.想清楚了再买，要分清自己是仿古地板两种消费类型中的哪一种，金象恒森地板更喜欢深识仿古地板的行家购买；3.要注意和家居装修风格的一体性，仿古地板更适合大平方米的住宅，家居面积太小不宜用仿古地板。'

In [8]:
#stop word

stopwords = pd.read_csv("./data/stopwords.txt",index_col=False,quoting=3,sep="\t",names=['stopword'],encoding='utf-8')
stopwords = stopwords['stopword'].values # put it into the array

In [9]:
stopwords

array(['!', '"', '#', ..., '07', '08', '09'], dtype=object)

In [10]:
#corpus
def preprocess_text(content_lines, sentences, category):
    for line in content_lines:
        try:
            sens = jieba.lcut(line)#cut the everyline into the word
            sens = filter(lambda x:len(x)>1,sens)#frequncy
            sens = filter(lambda x:x not in stopwords,sens)#stop
            sentences.append((" ".join(sens),category))#!!!!!
        except Exception as e:
            print(line)
            continue

sentences = []

preprocess_text(sport, sentences, 'sport')
preprocess_text(home,sentences, 'home')
preprocess_text(fashion,sentences, 'fashion')
preprocess_text(house, sentences, 'house')
preprocess_text(edu, sentences, 'edu')
preprocess_text(politic, sentences, 'politic')
preprocess_text(enter, sentences, 'entertainment')
preprocess_text(game,sentences, 'game')

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.955 seconds.
Prefix dict has been built succesfully.


In [11]:
#former dictionary built
#split the data set into different sets
from sklearn.model_selection import train_test_split
x, y = zip(*sentences)
train_data, test_data, train_target, test_target = train_test_split(x, y, random_state=1234)

In [12]:
#CNN
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

import numpy as np
import pandas
from sklearn import metrics
import tensorflow as tf

learn = tf.contrib.learn

FLAGS =None

MAX_DOCUMENT_LENGTH = 100
MIN_WORD_FREQUENCE = 2

EMBEDDING_SIZE = 20
#CORE???
N_FILTERS =10
WINDOW_SIZE =20
FILTER_SHAPE1 = [WINDOW_SIZE, EMBEDDING_SIZE] #20*20
FILTER_SHAPE2 = [WINDOW_SIZE, N_FILTERS] #20*10

#POOLING
POOLING_WINDOW = 4
POOLING_STRIDE = 2
n_words = 0

In [29]:
#LM
def cnn_model(feature, target):
    # word_feature_weight_matrix n_words * embedding_size
    #!!!!!!!!input2matrix[batch_size,seq_len,embedding_size]
    target =tf.one_hot(target, 15, 1, 0)
    #!!!!!!!Embedding
    word_vectors = tf.contrib.layers.embed_sequence(feature, vocab_size =n_words, embed_dim = EMBEDDING_SIZE, scope ='words')
    word_vectors = tf.expand_dims(word_vectors,3)
    
    #every layer has it own convolution and pooling
    #First Layer
    with tf.variable_scope('CNN_Layer1'):
        conv1 = tf.contrib.layers.convolution2d(
                word_vectors, N_FILTERS, FILTER_SHAPE1, padding = 'VALID')
        conv1 = tf.nn.relu(conv1)
        #maxpool
        pool1 =tf.nn.max_pool(
                conv1,
                ksize= [1,POOLING_WINDOW,1 ,1],
                strides =[1, POOLING_STRIDE,1 ,1],
                padding = 'SAME')
        #x.t
        pool1 = tf.transpose(pool1,[0,1,3,2])
    #Second Layer
    with tf.variable_scope('CNN_Layer2'):
        conv2 = tf.contrib.layers.convolution2d(pool1, N_FILTERS, FILTER_SHAPE2, padding= 'VALID')
        
        pool2 = tf.squeeze(tf.reduce_max(conv2, 1), squeeze_dims=[1])
    
    #The third fully connected
    logits = tf.contrib.layers.fully_connected(pool2, 15, activation_fn=None)#!!!!!!
    #the merely connect layer 
    loss =tf.losses.softmax_cross_entropy(target, logits)
    
    train_op =tf.contrib.layers.optimize_loss(
                loss,
                tf.contrib.framework.get_global_step(),
                optimizer ='Adam',
                learning_rate=0.01)
    return({
        'class': tf.argmax(logits,1),
        'prob': tf.nn.softmax(logits)
    },loss, train_op)

In [14]:
global n_words
# 处理词汇
vocab_processor = learn.preprocessing.VocabularyProcessor(MAX_DOCUMENT_LENGTH, min_frequency=MIN_WORD_FREQUENCE)
x_train = np.array(list(vocab_processor.fit_transform(train_data)))
x_test = np.array(list(vocab_processor.transform(test_data)))
n_words = len(vocab_processor.vocabulary_)
print('Total words: %d' % n_words)

Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Total words: 48058


In [15]:
#build a list by hand
cate_dic = {'sport':1, 'home':2, 'fashion':3, 'house':4, 'edu':5, 'politic':6, 'entertainment':7,'game':8}


In [16]:
#yingshe
train_target = map(lambda x:cate_dic[x], train_target)
test_target = map(lambda x:cate_dic[x], test_target)

In [17]:
y_train = pandas.Series(train_target)
y_test = pandas.Series(test_target)

In [30]:
# 构建模型
classifier = learn.SKCompat(learn.Estimator(model_fn=cnn_model))

# 训练和预测
classifier.fit(x_train, y_train, steps=1000)
y_predicted = classifier.predict(x_test)['class']
score = metrics.accuracy_score(y_test, y_predicted)
print('Accuracy: {0:f}'.format(score))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_master': '', '_eval_distribute': None, '_log_step_count_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_save_checkpoints_steps': None, '_evaluation_master': '', '_is_chief': True, '_session_config': None, '_keep_checkpoint_every_n_hours': 10000, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_environment': 'local', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3e5a223320>, '_device_fn': None, '_num_worker_replicas': 0, '_model_dir': '/tmp/tmp9rj211md', '_protocol': None, '_tf_random_seed': None, '_save_summary_steps': 100, '_task_id': 0, '_task_type': None, '_train_distribute': None, '_num_ps_replicas': 0}
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.EstimatorSpec. You can use the `estimator_spec` method to create an equivalent one.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow

In [31]:
# GRU
def rnn_model(features, target):
    
    # Convert indexes of words into embeddings.
    # This creates embeddings matrix of [n_words, EMBEDDING_SIZE] and then
    # maps word indexes of the sequence into [batch_size, sequence_length,
    # EMBEDDING_SIZE].
    word_vectors = tf.contrib.layers.embed_sequence(
            features, vocab_size=n_words, embed_dim=EMBEDDING_SIZE, scope='words')

    # Split into list of embedding per word, while removing doc length dim.
    # word_list results to be a list of tensors [batch_size, EMBEDDING_SIZE].
    word_list = tf.unstack(word_vectors, axis=1)

    # Create a Gated Recurrent Unit cell with hidden size of EMBEDDING_SIZE.
    cell = tf.contrib.rnn.GRUCell(EMBEDDING_SIZE)

    # Create an unrolled Recurrent Neural Networks to length of
    # MAX_DOCUMENT_LENGTH and passes word_list as inputs for each unit.
    _, encoding = tf.contrib.rnn.static_rnn(cell, word_list, dtype=tf.float32)

    # Given encoding of RNN, take encoding of last step (e.g hidden size of the
    # neural network of last step) and pass it as features for logistic
    # regression over output classes.
    target = tf.one_hot(target, 15, 1, 0)
    logits = tf.contrib.layers.fully_connected(encoding, 15, activation_fn=None)
    loss = tf.contrib.losses.softmax_cross_entropy(logits, target)

    # Create a training op.
    train_op = tf.contrib.layers.optimize_loss(
            loss,
            tf.contrib.framework.get_global_step(),
            optimizer='Adam',
            learning_rate=0.01)

    return ({
            'class': tf.argmax(logits, 1),
            'prob': tf.nn.softmax(logits)
    }, loss, train_op)

In [32]:
model_fn = rnn_model
classifier = learn.SKCompat(learn.Estimator(model_fn=model_fn))

# Train and predict
classifier.fit(x_train, y_train, steps=1000)
y_predicted = classifier.predict(x_test)['class']
score = metrics.accuracy_score(y_test, y_predicted)
print('Accuracy: {0:f}'.format(score))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_master': '', '_eval_distribute': None, '_log_step_count_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_save_checkpoints_steps': None, '_evaluation_master': '', '_is_chief': True, '_session_config': None, '_keep_checkpoint_every_n_hours': 10000, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_environment': 'local', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3e597fd588>, '_device_fn': None, '_num_worker_replicas': 0, '_model_dir': '/tmp/tmp_mt7gefp', '_protocol': None, '_tf_random_seed': None, '_save_summary_steps': 100, '_task_id': 0, '_task_type': None, '_train_distribute': None, '_num_ps_replicas': 0}
Instructions for updating:
Use tf.losses.softmax_cross_entropy instead. Note that the order of the logits and labels arguments has been changed.
Instructions for updating:

Future major versions of TensorFlow will allow gradients